## Module Loading

In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import *
from sklearn.linear_model import LogisticRegression
from tqdm.notebook import tqdm
import spacy
from nltk.stem.snowball import SnowballStemmer

## Data Loading

In [4]:
train = pd.read_csv("MBTI_train.csv",names=['mbti','text'], header=None, encoding="ISO 8859-1")
train.head()

,mbti,text
0,INTP,say process model list like subscriber channel...
1,INFJ,upon much manipulate retail finish like sacrif...
2,INFJ,fit yes certain bff social feel goal go know n...
3,INTJ,complete love within someone ideal joke solvea...
4,ENTJ,public strictly thing person x question person...


In [6]:
test = pd.read_csv("MBTI_test.csv",names=['text'], header=None, encoding="ISO 8859-1")
test.head()

,text
0,get accept ya bite well stop important open lo...
1,offer rebel something war people friend block ...
2,soulmates contradiction easy basic recurrence ...
3,run nature q test let sound sarcastically irri...
4,hour fast fast suspend see strict wampum eight...


설명변수 (X) : 400개의 단어 <br>
예측변수 (y) : MBTI

In [7]:
#설명변수
X = train['text']
#예측변수
y = train['mbti'] 

## EDA

1. 라벨 개수 확인

In [8]:
print(f"본 데이터셋에는 {len(train['mbti'].unique())}개 타입의 MBTI가 있다")

본 데이터셋에는 16개 타입의 MBTI가 있다


2. 각 라벨별 비율 확인

In [9]:
train['mbti'].value_counts()

INTP    17690
INTJ    16079
INFJ    10765
INFP     8444
ENTP     8303
ENFP     4338
ISTP     2394
ENTJ     2106
ENFJ     1024
ISTJ      878
ESTP      814
ISFP      620
ISFJ      460
ESFP      252
ESFJ      119
ESTJ       71
Name: mbti, dtype: int64

3. 결측치 확인

In [10]:
train.isnull().sum()

mbti    0
text    0
dtype: int64

4. 데이터 중복 여부 확인

In [15]:
train['text'].nunique() == len(train['text']) # 중복 없음

True

5. train 데이터와 test 데이터 크기 확인

In [16]:
print ("train 데이터 : ", train.shape)
print ("test 데이터 : ", test.shape)

train 데이터 :  (74357, 2)
test 데이터 :  (9337, 1)


6. 각 자리별 빈도수 확인

In [148]:
# 첫번째 글자
first = []
for i in range(len(train)):
    first.append(train['mbti'][i][0])
first = pd.DataFrame(first)
first[0].value_counts()

I    57330
E    17027
Name: 0, dtype: int64

In [149]:
# 두번째 글자
second = []
for i in range(len(train)):
    second.append(train['mbti'][i][1])
second = pd.DataFrame(second)
second[0].value_counts()

N    68749
S     5608
Name: 0, dtype: int64

In [150]:
# 세번째 글자
third = []
for i in range(len(train)):
    third.append(train['mbti'][i][2])
third = pd.DataFrame(third)
third[0].value_counts()

T    48335
F    26022
Name: 0, dtype: int64

In [151]:
# 네번째 글자
fourth = []
for i in range(len(train)):
    fourth.append(train['mbti'][i][3])
fourth = pd.DataFrame(fourth)
fourth[0].value_counts()

P    42855
J    31502
Name: 0, dtype: int64

## Split data

In [26]:
X = train['text'] # features
y = train['mbti']  # labels
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

## Modeling_1 : only TfidfVectorizer

##### TfidfVectorizer_자연어처리
- CountVectorizer이 의미가 없지만 빈도가 높은 단어의 가중치 증가의 단점 해결
- Tf(Term Frequency) : 하나의 무엇에서 특정 단어가 등장하는 횟수
- Idf(Inverse Document Frequency) : 특정 단어가 몇개의 text에서 등장하는지의 역수
- 역수 ) 어려 문서에 의미 없이 사용되는 단어의 가중치를 줄이기 위함

-->Tf-idf 수치 = Tf * Idf

In [18]:
#벡터화
tfidf = TfidfVectorizer()
    
# 훈련 데이터 벡터화
X_train_tfidf = tfidf.fit_transform(X_train)

1. Grid Search
- hyperparameter 'C' 변경

In [21]:
clf = LinearSVC()

#정확도 기준
cv = GridSearchCV(clf, {'C': [0.1,0.3, 0.5,1.0]}, scoring ="accuracy")

text_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',cv)])
text_clf.fit(X_train, y_train)

C = cv.best_estimator_.C

In [23]:
print("The best Parameter C is ",C)

The best Parameter C is  0.3


2. Grid Search 결과를 바탕으로 Validation Accuracy 확인

In [24]:
text_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC(C=0.3))])
text_clf.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC(C=0.3))])

In [27]:
# valid 데이터의 mbti 예측
predictions = text_clf.predict(X_valid)

In [28]:
# valid data에서의 정확도
accuracy_score(predictions, y_valid)

0.7939080150618613

2-1. MBTI 첫번째 글자의 정확도 

In [43]:
y_valid_list = list(y_valid)

In [45]:
num = 0
for i in range(len(y_valid)):
    if y_valid_list[i][0] == predictions[i][0]:
        num+=1
print("SVM 모델 MBTI 첫번째 글자의 정확도 : ", num/len(y_valid))

SVM 모델 MBTI 첫번째 글자의 정확도 :  0.9221355567509414


In [47]:
num = 0
for i in range(len(y_valid)):
    if y_valid_list[i][1] == predictions[i][1]:
        num+=1
print("SVM 모델 MBTI 두번째 글자의 정확도 : ", num/len(y_valid))

SVM 모델 MBTI 두번째 글자의 정확도 :  0.9616057019903174


In [48]:
num = 0
for i in range(len(y_valid)):
    if y_valid_list[i][2] == predictions[i][2]:
        num+=1
print("SVM 모델 MBTI 세번째 글자의 정확도 : ", num/len(y_valid))

SVM 모델 MBTI 세번째 글자의 정확도 :  0.9178321678321678


In [49]:
num = 0
for i in range(len(y_valid)):
    if y_valid_list[i][3] == predictions[i][3]:
        num+=1
print("SVM 모델 MBTI 세번째 글자의 정확도 : ", num/len(y_valid))

SVM 모델 MBTI 세번째 글자의 정확도 :  0.8813878429263045


3. 모든 train 데이터를 이용하여 학습 후 test 데이터 예측

In [50]:
# 모든 설명변수 데이터 X 자연어처리
X_tfidf = tfidf.fit_transform(X)

In [51]:
svc_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC(C=0.3))])
svc_clf.fit(X, y)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC(C=0.3))])

In [54]:
predictions_svc = svc_clf.predict(test['text'])

In [72]:
test_pred = pd.DataFrame(predictions_svc)

## Modeling_2 : 무의미한 단어 제거, 표제어 추출

In [64]:
# StopWord 확인
nlp = spacy.load('en_core_web_sm')
print("불러온 Stopword (무의미한 단어)", len(nlp.Defaults.stop_words))
def remove_stopwords(s):
  new_s = []
  for word in s.split():
    if not nlp.vocab[word].is_stop:
      new_s.append(word)
  return ' '.join(new_s)

불러온 Stopword (무의미한 단어) 326


In [65]:
# Lemmatization(표제어 추출 : 단어로부터 표제어 찾기)

s_stemmer = SnowballStemmer(language='english')
def replace_stemwords(s):
  new_s = []
  for word in s.split():
    new_s.append(s_stemmer.stem(word))
  return ' '.join(new_s)

In [70]:
tqdm.pandas()

C:\Users\jisuj\anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [69]:
train

,mbti,text
0,INTP,say process model list like subscriber channel...
1,INFJ,upon much manipulate retail finish like sacrif...
2,INFJ,fit yes certain bff social feel goal go know n...
3,INTJ,complete love within someone ideal joke solvea...
4,ENTJ,public strictly thing person x question person...
...,...,...
74352,INTP,get consequence process life back kind great b...
74353,ENFP,mundane really right vibe natural u conscious ...
74354,INFP,diva message remember practical lot absorb tel...
74355,INTJ,increase impressive group make recognize get a...


In [71]:
# StopWord 제거
train.text = train.text.progress_apply(lambda x: remove_stopwords(replace_stemwords(x)))

In [73]:
#test에서도 표제어 추출
test.text = test.text.progress_apply(lambda x: remove_stopwords(replace_stemwords(x)))

In [74]:
X = train.text
y= train.mbti

In [115]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [116]:
text_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC(C=0.3))])
text_clf.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC(C=0.3))])

In [117]:
# valid 데이터의 mbti 예측
predictions_val = text_clf.predict(X_valid)

In [118]:
# valid data에서의 정확도
accuracy_score(predictions_val, y_valid)

0.7942442173211404

2-1. MBTI 첫번째 글자의 정확도 

In [119]:
y_valid_list = list(y_valid)

In [120]:
num = 0
for i in range(len(y_valid)):
    if y_valid_list[i][0] == predictions_val[i][0]:
        num+=1
print("SVM 모델 MBTI 첫번째 글자의 정확도 : ", num/len(y_valid))

SVM 모델 MBTI 첫번째 글자의 정확도 :  0.9224717590102205


In [123]:
num = 0
for i in range(len(y_valid)):
    if y_valid_list[i][1] == predictions_val[i][1]:
        num+=1
print("SVM 모델 MBTI 두번째 글자의 정확도 : ", num/len(y_valid))

SVM 모델 MBTI 두번째 글자의 정확도 :  0.9625470683162991


In [121]:
num = 0
for i in range(len(y_valid)):
    if y_valid_list[i][2] == predictions_val[i][2]:
        num+=1
print("SVM 모델 MBTI 세번째 글자의 정확도 : ", num/len(y_valid))

SVM 모델 MBTI 세번째 글자의 정확도 :  0.9180338891877353


In [122]:
num = 0
for i in range(len(y_valid)):
    if y_valid_list[i][3] == predictions_val[i][3]:
        num+=1
print("SVM 모델 MBTI 네번째 글자의 정확도 : ", num/len(y_valid))

SVM 모델 MBTI 네번째 글자의 정확도 :  0.8789671866594944


정확도 | Modeling_1 | Modeling_2
---|---|---
첫번째|92.21|92.24
두번째|96.16|96.25
세번째|91.78|91.8
네번째|88.13|87.89

- 0.1이상 차이나는 자료 대체

3. 모든 train 데이터를 이용하여 학습 후 test 데이터 예측

In [50]:
# 모든 설명변수 데이터 X 자연어처리
X_tfidf = tfidf.fit_transform(X)

In [85]:
svc_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC(C=0.3))])
svc_clf.fit(X, y)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC(C=0.3))])

In [86]:
predictions_svc_2 = svc_clf.predict(test['text'])

In [87]:
test_pred_2 = pd.DataFrame(predictions_svc_2)

In [136]:
test_pred.head()

,0
0,ENFP
1,ENTP
2,INTJ
3,INTJ
4,INTJ


In [137]:
test_pred_2.head()

,0
0,ENFP
1,ENTP
2,INTJ
3,INTJ
4,INTJ


## 최종 데이터프레임 생성
- test_pred의 첫번째, 세번째, 네번째 글자와 test_pred_2의 두번째 글자 합침

In [126]:
list_combine=[]

In [127]:
for i in range(len(test_pred)):
    list = []
    list.append(test_pred[0][i][0])
    list.append(test_pred_2[0][i][1])
    list.append(test_pred[0][i][2:])
    combine = ''.join(list)
    list_combine.append(combine)

In [128]:
list_df = pd.DataFrame(list_combine)

In [129]:
list_df.to_csv("Final_result_team11.csv", index=False, header = False)

In [153]:
!pip freeze > requirements.txt